In [5]:
pip install fpdf

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40712 sha256=f74250b13bf22e75334899ca60f413b02a01edc4c279da2587162d639e2934c5
  Stored in directory: c:\users\gnanalistaincom\appdata\local\pip\cache\wheels\65\4f\66\bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd

# Cargar el archivo en un DataFrame
df_cargos = pd.read_excel("data_cargos.xlsx")

# Mostrar las primeras filas para visualizar la estructura
df_cargos.head()

,Cargo,GRUPO,Informe de ventas -gerencial,SI gerencial CASO.csv,SI Objetivos CASO.csv,Informe de ventas -FoodService,Informe de ventas -Kam/Mkt,Informe Gestión Zonales,Informe Sell out,Scantrack,...,Visualizaciones de Reportes,Reporte Clientes sin asignar,Golden Store Premezcla,Golden Store Tortillas,Golden Store Mascotas,Reporte SAC,Politica de Precio Revenue,Reporte Indicadores Lanzamientos,Reporte productividad 2022,Informe Oportunidades por Segmentos 23
0,ASISTENTE DE MARKETING,ASISTENTE MKT,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,ANALISTA CPFR,SUPPLY Y DP,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,ANALISTA COSTOS,VARIOS INF VENTAS,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,JEFE ZONAL,JEFE ZONAL,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
4,COORDINADOR,COORDIN_TRADE MKT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Iterar sobre cada fila y construir el texto basado en los reportes a los que tiene acceso el cargo

In [3]:
def row_to_text(row):
    # Extraer el cargo y el grupo
    cargo = row["Cargo"]
    grupo = row["GRUPO"]
    
    # Extraer los reportes a los que tiene acceso (columnas con valor 1)
    reportes_acceso = row[row == 1].index.tolist()
    
    # Construir el texto
    if reportes_acceso:
        reportes_str = ', '.join(reportes_acceso[:-1])
        if len(reportes_acceso) > 1:
            reportes_str += " y " + reportes_acceso[-1]
        return f"Del Grupo {grupo}, para el Cargo {cargo}, tiene acceso a los reportes {reportes_str}."
    else:
        return f"Del Grupo {grupo}, para el Cargo {cargo}, no tiene acceso a ningún reporte."

# Visualizar el texto para la primera fila
row_to_text(df_cargos.iloc[0])


'Del Grupo ASISTENTE MKT, para el Cargo ASISTENTE DE MARKETING, tiene acceso a los reportes Informe de ventas -gerencial, Informe de ventas -Kam/Mkt, Informe Sell out, Scantrack, Scorecard Pascua, Bresler, Inf Precios, Informe OSA, Reporte Halloween, Reporte Facing (depende de Categoria), Oportunidades por Segmentos, Cobertura por cadenas, Informe Facing Galletas Agosto 2021, Informe Cornershop, Informe de Precios, Informe Ecommerce, OSA semanal, Osa Cadenas, Informe Facing Arenas 2021, Informe Facing Postres Octubre 2021, Reporte Fill Rate y OTIF, Sell out.csv y Reporte productividad 2022.'

#### Generaremos el texto para todas las filas y luego escribiremos ese texto en un archivo PDF

- Iterar sobre todas las filas del DataFrame y generar el texto para cada fila utilizando la función row_to_text.
- Utilizar la biblioteca fpdf2 para escribir cada texto en un archivo PDF.
- Guardar el archivo PDF.


In [7]:
def sanitize_text(text):
    """Reemplaza los caracteres no admitidos por caracteres similares o los elimina."""
    replacements = {
        "–": "-",  # Reemplazar guiones largos por guiones cortos
        "’": "'",  # Reemplazar comillas simples curvas por comillas simples rectas
    }
    for original, replacement in replacements.items():
        text = text.replace(original, replacement)
    return text.encode("latin-1", "ignore").decode("latin-1")

# Crear instancia de PDF nuevamente
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()
pdf.set_font("Arial", size=12)

# Iterar sobre las filas y generar el texto
for _, row in df_cargos.iterrows():
    text = sanitize_text(row_to_text(row))
    pdf.multi_cell(0, 10, text)

# Guardar el archivo PDF
pdf_output_path = "reportes_acceso_sanitized.pdf"
pdf.output(pdf_output_path)

pdf_output_path


'reportes_acceso_sanitized.pdf'